In [29]:
from os import times

import pandas as pd
import numpy as np
from datetime import datetime
import pymysql
from datetime import datetime
import time

import warnings
warnings.filterwarnings('ignore')

In [30]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

In [31]:
sql = '''
select tprm.order_id, om.merchant_name, tprm.order_number, tprm.order_create_time, tprm.total_receivable, tprm.contract_price, tprm.all_rental, tprm.rent_received, tprm.purchase_amount, too.real_pay_money, tod.new_actual_money
from db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order om on tprm.order_id = om.id
left join db_digua_business.t_order_details tod on tod.order_id = tprm.order_id
left join (select order_id, sum(real_pay_money) real_pay_money from db_digua_business.t_order_out where status!=1 group by order_id) too on too.order_id=tprm.order_id
-- where date_format(tprm.order_create_time, '%Y-%m')>='2023-01'
-- and date_format(tprm.order_create_time, '%Y-%m')<='2024-12'
'''
df = query(sql)
df.drop(df[df['merchant_name'].str.contains(pat='探路者', regex=False) == True].index, inplace=True)

In [32]:
sql1 = '''
select ybt.order_id,
om.create_time, om.merchant_name, ybt.add_time, om.order_number, ybt.`status`, om.has_actual, ybt.money, ybt.assign_user_name, ybt.completion_amount, tprm.purchase_amount, tod.new_actual_money
,tprm.order_create_time, tprm.total_receivable, tprm.contract_price, tprm.all_rental, tprm.rent_received, too.real_pay_money
from db_rent.ya_buyout_task ybt
left join db_digua_business.t_order om on om.id=ybt.order_id
left join db_digua_business.t_postlease_receivables_monitoring tprm on ybt.order_id=tprm.order_id
left join db_digua_business.t_order_details tod on tod.order_id=tprm.order_id
left join (select order_id, sum(real_pay_money) real_pay_money from db_digua_business.t_order_out where status!=1 group by order_id) too on too.order_id=tprm.order_id
where date_format(ybt.end_time, '%Y-%m-%d')>='2025-02-01'
and date_format(ybt.end_time, '%Y-%m-%d')<='2025-02-15'
'''
df1 = query(sql1)
df1.drop(df1[df1['merchant_name'].str.contains(pat='探路者', regex=False) == True].index, inplace=True)

In [33]:
sql_out = '''
select too.order_id, ymos.sort
from (select order_id, min(pay_date) as pay_date from db_digua_business.t_order_out where status=3 group by order_id) too
left join db_rent.ya_merchant_order_stages ymos on too.order_id=ymos.order_id
where date_format(ymos.refund_date, '%Y-%m')=date_format(too.pay_date, '%Y-%m')
'''
df_out = query(sql_out)

In [34]:
df_out.groupby('order_id').agg({'sort': 'count'}).sort_values('sort', ascending=False)

,sort
order_id,
21960,1
139297,1
209965,1
210091,1
210347,1
...,...
1739122,1
1752376,1
1759691,1


In [35]:
df_md = pd.read_excel('F:/需求/廖志彧需求/2月1-15提前买断订单.xlsx', sheet_name='提前买断')
df_md_merge = df_md.merge(df, left_on='单号', right_on='order_number', how='left').merge(df_out, on='order_id', how='left')

应收租金：买断订单的剩余期数未付租金+买断尾款

已收租金：已收租金+买断尾款

毛利率：(已收租金-采购成本)/采购成本

In [36]:
# 提前买断
df_md_merge['real_pay_money'] = df_md_merge.real_pay_money.fillna(0)
df_md_merge.loc[:, '已收租金'] = df_md_merge.rent_received + df_md_merge.real_pay_money
df_md_merge.loc[:, '优惠金额'] = df_md_merge.new_actual_money-df_md_merge.已收租金
# df_md_merge.loc[:, '应收租金'] = np.where(df_md_merge.rent_received>df_md_merge.all_rental, df_md_merge.contract_price-df_md_merge.all_rental, df_md_merge.应收租金)
df_md_merge.loc[:, '优惠比例'] = (df_md_merge.优惠金额/df_md_merge.purchase_amount).map(lambda x: format(x, '.2%'))
df_md_merge.loc[:, '优惠租金比例'] = (df_md_merge.优惠金额/df_md_merge.已收租金).map(lambda x: format(x, '.2%'))
df_md_merge.loc[:, '毛利率'] = ((df_md_merge.已收租金-df_md_merge.purchase_amount)/df_md_merge.purchase_amount).map(lambda x: format(x, '.2%'))
df_md_merge = df_md_merge[['单号', 'sort', 'new_actual_money', '优惠金额', '已收租金', 'purchase_amount', '优惠比例', '优惠租金比例', '毛利率']].rename(columns={ 'new_actual_money':'签约价', 'purchase_amount': '采购成本', 'sort': '买断期数'})

In [37]:
order_list = df_md_merge.单号.to_list()

In [38]:
# 任务池
df1_finish = df1[df1.status==3]
df1_finish = df1_finish.merge(df_out, on='order_id', how='left')
# df1_finish = df1_finish[~df1_finish.order_number.isin(order_list)]
df1_finish['real_pay_money'] = df1_finish.real_pay_money.fillna(0)
# df1_finish.loc[:, '应收租金'] = df1_finish.all_rental-df1_finish.rent_received+(df1_finish.contract_price-df1_finish.all_rental)
# df1_finish.loc[:, '应收租金'] = np.where(df1_finish.rent_received>df1_finish.all_rental, df1_finish.contract_price-df1_finish.all_rental, df1_finish.应收租金)
df1_finish.loc[:, '已收租金'] = df1_finish.rent_received + df1_finish.real_pay_money
df1_finish.loc[:, '优惠金额'] = df1_finish.new_actual_money-df1_finish.已收租金
df1_finish.loc[:, '优惠比例'] = (df1_finish.优惠金额/df1_finish.purchase_amount).map(lambda x: format(x, '.2%'))
df1_finish.loc[:, '优惠租金比例'] = (df1_finish.优惠金额/df1_finish.已收租金).map(lambda x: format(x, '.2%'))
df1_finish.loc[:, '毛利率'] = ((df1_finish.已收租金-df1_finish.purchase_amount)/df1_finish.purchase_amount).map(lambda x: format(x, '.2%'))
df1_finish = df1_finish[['order_number', 'sort', 'new_actual_money', '优惠金额', '已收租金', 'purchase_amount', '优惠比例', '优惠租金比例', '毛利率']].rename(columns={'order_number':'单号', 'new_actual_money':'签约价', 'purchase_amount': '采购成本', 'sort': '买断期数'})

In [39]:
t_day = datetime.today().strftime('%Y%m%d%H')

In [40]:
with pd.ExcelWriter(f'F:/需求/廖志彧需求/提前买断数据1_{t_day}.xlsx', engine='xlsxwriter') as writer:
    df_md_merge.to_excel(writer, sheet_name='提前买断', index=False)
    df1_finish.to_excel(writer, sheet_name='任务池已完成', index=False)